# A script to remotely download NAIP imagery  

In [107]:
import re
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.keys import Keys
import time
import os

### Create a look-up dataframe pairing a county with it's NAIP identifier

In [67]:
# 's' is an exerpt from 'NAIP File Name.pdf' at https://nrcs.app.box.com/v/naip/file/168958604571
s = "NM001BERNALILLONM005CHAVESNM007COLFAXNM009CURRYNM011DE BACANM013DONA ANANM015EDDYNM017GRANTNM019GUADALUPENM021HARDINGNM023HIDALGONM025LEANM027LINCOLNNM029LUNANM033MORANM035OTERONM037QUAYNM039RIO ARRIBANM041ROOSEVELTNM043SANDOVALNM045SAN JUANNM047SAN MIGUELNM049SANTA FENM051SIERRANM053SOCORRONM055TAOSNM057TORRANCENM059UNIONNM061VALENCIA"

# Parse the string into a pandas DataFrame that will serve as a look-up table for downloading the imagery.
nm_counties = s.split('NM')
cty_list = []

for county in nm_counties:
    counties = re.split('(\d+)', county)
    cty_list.append(counties)

mckinley = ['','031', 'MCKINLEY']
cty_list.append(mckinley)

df = pd.DataFrame(sorted(cty_list), columns=["Index", "NAIP_ID", "County"])
df2 = df.iloc[1:]
df2


,Index,NAIP_ID,County
1,,001,BERNALILLO
2,,005,CHAVES
3,,007,COLFAX
4,,009,CURRY
5,,011,DE BACA
6,,013,DONA ANA
7,,015,EDDY
8,,017,GRANT
9,,019,GUADALUPE
10,,021,HARDING


In [68]:
counties_of_interest = ["QUAY", "CURRY", "DE BACA", "CHAVES", "LINCOLN", "ROOSEVELT", "EDDY", "LEA", "SAN JUAN",
                        "RIO ARRIBA", "SANDOVAL", "MCKINLEY"]

In [70]:
#Select counties of interest out of the look-up dataframe and store in a new dataframe.
sel_cty = df[df['County'].isin(counties_of_interest)]
sel_cty['NAIP_ID']

2     005
4     009
5     011
7     015
12    025
13    027
15    031
18    037
19    039
20    041
21    043
22    045
Name: NAIP_ID, dtype: object

In [113]:
# Initialize webdriver
dl_loc = <<path to download location>>
options = Options()
prefs = {'download.default_directory' : dl_loc, 'useAutomationExtension': False, 'forceDevToolsScreenshot': True,
        'args': ['--disable-infobars', "--disable-extensions"]}
options.add_experimental_option('prefs', prefs)
driver = webdriver.Chrome(chrome_options=options)
driver.get("https://nrcs.app.box.com/v/naip/folder/18143730559")

In [112]:
def downloads_done():
    for i in os.listdir(dl_loc):
        if ".crdownload" in i:
            print("Downloading...")
            time.sleep(300)
            downloads_done()

In [ ]:
for id in sel_cty['NAIP_ID']:
    try:
        imagery = driver.find_element_by_link_text('ortho_1-1_1n_s_nm{}_2016_1.zip'.format(id))
        imagery.click()

        time.sleep(5)

        download = driver.find_element_by_xpath('/html/body/div[1]/span/div/span/div/div[1]/div[2]/button/span/span')
        download.click()

        driver.back()
        downloads_done()
    except NoSuchElementException:
        next_page = driver.find_element_by_xpath('/html/body/div[1]/span/div/main/div/div/div[2]/div[2]/div/a')
        next_page.send_keys(Keys.END)
        time.sleep(2)
        next_page.click()

        time.sleep(2)

        imagery = driver.find_element_by_link_text('ortho_1-1_1n_s_nm{}_2016_1.zip'.format(id))
        imagery.click()

        time.sleep(5)

        download = driver.find_element_by_xpath('/html/body/div[1]/span/div/span/div/div[1]/div[2]/button/span/span')
        download.click()

        driver.back()
        driver.back()
        downloads_done()
        
    